In [189]:
import spacy
import json
import os
from nltk.corpus import stopwords, words
import pandas as pd
import gensim
import gensim.corpora as corpora
import re
import contractions
import string

In [190]:
files = []
subdir = tuple(os.walk("../MergedData/merged_movie_jsons/1997"))
for file_name in tuple(subdir)[0][-1]:
    with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
        files.append(json.load(f))

In [191]:
corpus = []
my_pos = set(["ADJ"])
nlp = spacy.load("en_core_web_lg") #python -m spacy download en_core_web_lg


for file in files[::-1]:
    content = [review["content"] for review in file["reviews"]]
    for review in content:
        s = nlp(review)
        clean = []
        for word in s:
            if word.pos_ in my_pos:
                clean.append(str(word.lemma_))
        corpus.append(" ".join(clean))
    break

In [192]:
# corpus
# remove non english words again
# remove numbers
# remove words shorted than 3 letters
# remove punctuations
# print(type(stopwords.words("english")))

In [193]:
with open("../stopwords.json", "r") as f:
    common = json.load(f)
    common.extend(stopwords.words("english"))
    common = set(common)


eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word not in common and word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [194]:
proc_data = [clean(movie_revs).split() for movie_revs in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [195]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict,  normalize=True, dictionary=input_dict, slope=2.5, smartirs="Lfc")
vector = model[input_corpus]

In [196]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gensim/models/tfidfmodel.py:180: RuntimeWarning: Mean of empty slice.
  return (1 + np.log2(tf)) / (1 + np.log2(tf.mean(axis=0)))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [197]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [198]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df["TF-IDF"].between(0.8 * df["TF-IDF"].median(), 1.05 * df["TF-IDF"].median(), inclusive=True))
    # pass

In [199]:
# kmeans = KMeans(n_clusters=100, init="k-means++", n_init=10000)
# vect = TfidfVectorizer(stop_words="english")
# features = vect.fit_transform([ " ".join(data) for data in proc_data])
# kmeans.fit(features)

In [200]:
# df2 = pd.DataFrame(corpus)
# df2["clusters"] = kmeans.labels_
# df2

In [201]:
# print("Cluster centrojds: \n")
# order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vect.get_feature_names_out()
# for i in range (100):
#     print("cluster %d:" % i)
#     for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
#         print(' %s ' % terms [j])

In [202]:
med = df["TF-IDF"].median()
mean = df["TF-IDF"].mean()
vibes = set()
for k, v in d.items():
    if 0.95 *  med <= v <= 1.05 * med:
        vibes.add(k)


In [203]:
cnt = {}
for sent in corpus:
    sent = clean(sent)
    for word in sent.split():
        cnt[word] = cnt.get(word, 0) + 1

cnt = {k: v for k, v in sorted(cnt.items(), key=lambda item: item[1], reverse=True)}

for c in cnt:
    print(c, cnt[c])

real 830
well 520
rich 471
romantic 455
whole 360
historical 357
long 353
wonderful 336
high 292
titanic 292
least 290
tragic 283
emotional 260
worth 239
epic 222
fictional 217
top 216
sure 209
human 197
wrong 194
boring 194
enough 190
visual 176
unsinkable 173
accurate 166
impressive 162
hard 161
famous 160
able 156
awful 151
realistic 147
awesome 147
social 143
technical 142
cheesy 138
classic 136
popular 131
dramatic 127
horrible 125
powerful 122
superb 121
spectacular 119
nice 112
predictable 109
stunning 108
believable 107
cinematic 103
teenage 103
deep 103
grand 101
early 99
magnificent 97
entertaining 97
cold 96
expensive 96
outstanding 95
close 94
successful 92
next 90
memorable 87
cool 85
wealthy 84
easy 84
gone 83
personal 82
sorry 80
touching 79
timeless 78
exciting 77
worthy 77
free 76
present 76
corny 76
upper 75
enjoyable 75
gorgeous 75
complete 74
sappy 71
alive 70
late 69
evil 69
overall 68
unbelievable 68
nude 67
ridiculous 67
dead 66
talented 66
interested 65
impossib